In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from mpl_toolkits.mplot3d import Axes3D
import netCDF4
import numpy
import os
import pandas
from PIL import Image
import random
import readline
import scipy
from scipy import fft
from scipy import linalg
from scipy import stats
from scipy.stats import poisson, ttest_ind
import seaborn
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import normalize
from statsmodels.tsa.ar_model import AutoReg
import xarray as xr

In [2]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExTraTrack/rcp_files/'

In [3]:
# Open File
def Create_DF(File):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data
    for Line in Data:
        Rows.append(Line.strip())
    Storm_Code = []
    Storm_List_Orig = []
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            Code = Rows[i][41:45]
            Storm_List_Orig.append(Code)
        else:
            Storm_Code.append(Code)
    Array = numpy.zeros((13, len(Rows)-len(Storm_List_Orig)))
    Time = []
    k = -1
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            k += 1
        else:
            l = len(Rows[i]) - 100
#            if int(Rows[i][84+l:88+l]) != 2071 and int(Rows[i][84+l:88+l]) != 2072:
            Array[0][i-k-1] = float(Rows[i][0:6+l])
            Array[1][i-k-1] = float(Rows[i][8+l:14+l])
            Array[2][i-k-1] = float(Rows[i][17+l:24+l])
            Array[3][i-k-1] = float(Rows[i][26+l:31+l])
            Array[4][i-k-1] = float(Rows[i][34+l:41+l])
            Array[5][i-k-1] = float(Rows[i][44+l:51+l])
            Array[6][i-k-1] = float(Rows[i][54+l:61+l])
            Array[7][i-k-1] = float(Rows[i][64+l:71+l])
            Array[8][i-k-1] = float(Rows[i][74+l:81+l])
            Time.append(datetime.datetime(year=int(Rows[i][84+l:88+l]), month=int(Rows[i][90+l:92+l]), \
            day=int(Rows[i][94+l:96+l]), hour=int(Rows[i][98+l:100+l])))
#
# Create DataFrame to Store Data
    DF_All = pandas.DataFrame({"Orig Code": Storm_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "Dist(m)": Array[4], "Angle": Array[5], "B": Array[6], "VLT": Array[7], "VUT": Array[8], \
    "Orig Time(Z)": Time})
#
# Remove -999.0 Data Points
    DF = DF_All[(DF_All["B"] > -728) & (DF_All["VLT"] > -728) & (DF_All["VUT"] > -728)].reset_index()
    return (DF, Storm_List_Orig)

In [4]:
# Combine DataFrames
def Combine_DF(DF_A, DF_B, DF_C, Orig_List_A, Orig_List_B, Orig_List_C, Model):
    if Model == "Control":
        Year_Start = 1900
    elif Model == "RCP45":
        Year_Start = 2000
    elif Model == "RCP85":
        Year_Start = 2100
#        Year_Diff_A = Year_Start - 1985
#        Year_Diff_B = Year_Start - 1985 + 30
#        Year_Diff_C = Year_Start - 1985 + 60
    Origi_List_A = []
    Origi_List_B = []
    Origi_List_C = []
    New_List_A = []
    New_List_B = []
    New_List_C = []
    New_Time = []
    New_Code = []
    ABC = []
    Orig_Combine = []
    New_Combine = []
# Dataset A
    Year = Year_Start
    Orig_Year = DF_A["Orig Time(Z)"][0].year
    Year_Diff_A = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_A)):
        try:
            Code, Year, Count = Update_Code(DF_A, Orig_List_A[k], Year_Diff_A, Year, Count)
            Origi_List_A.append(Orig_List_A[k])
            New_List_A.append(Code)
        except:
            Kinen = 728
# Dataset B
    Year += 1
    Orig_Year = DF_B["Orig Time(Z)"][0].year
    Year_Diff_B = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_B)):
        try:
            Code, Year, Count = Update_Code(DF_B, Orig_List_B[k], Year_Diff_B, Year, Count)
            Origi_List_B.append(Orig_List_B[k])
            New_List_B.append(Code)
        except:
            Kinen = 728
# Dataset C
    Year += 1
    Orig_Year = DF_C["Orig Time(Z)"][0].year
    Year_Diff_C = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_C)):
        try:
            Code, Year, Count = Update_Code(DF_C, Orig_List_C[k], Year_Diff_C, Year, Count)
            Origi_List_C.append(Orig_List_C[k])
            New_List_C.append(Code)
        except:
            Kinen = 728
#
# Combine Into New Time and Codes List
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_A, Origi_List_A, New_List_A, Year_Diff_A, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "A")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_B, Origi_List_B, New_List_B, Year_Diff_B, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "B")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_C, Origi_List_C, New_List_C, Year_Diff_C, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "C")
#
# Concatenate Other Variables
    Vars = ["Lon", "Lat", "SLP(hPa)", "Winds(m/s)", "Dist(m)", "Angle", "B", "VLT", "VUT"]
    Total_Length = len(New_Code)
    Array = numpy.zeros((9,Total_Length))
    for n in range(len(Vars)):
        List = []
        for k in range(len(DF_A)):
            if (DF_A["Orig Time(Z)"][k].year != 2071) and (DF_A["Orig Time(Z)"][k].year != 2072):
                List.append(DF_A[Vars[n]][k])
        for k in range(len(DF_B)):
            if (DF_B["Orig Time(Z)"][k].year != 2071) and (DF_B["Orig Time(Z)"][k].year != 2072):
                List.append(DF_B[Vars[n]][k])
        for k in range(len(DF_C)):
            if (DF_C["Orig Time(Z)"][k].year != 2071) and (DF_C["Orig Time(Z)"][k].year != 2072):
                List.append(DF_C[Vars[n]][k])
        Array[n] = List
#        Array[n] = numpy.concatenate((DF_A_Var[Vars[n]], DF_B_Var[Vars[n]], DF_C_Var[Vars[n]]))
#
# Create DataFrame to Store Data
    DF_Combine = pandas.DataFrame({"Code": New_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "B": Array[6], "VLT": Array[7], "VUT": Array[8], "Time(Z)": New_Time})
    Codes_DF = pandas.DataFrame({"ABC": ABC, "Orig Code": Orig_Combine, "New Code": New_Combine})
    Year_Diffs = numpy.array([Year_Diff_A, Year_Diff_B, Year_Diff_C])
    return (DF_Combine, Codes_DF, Year_Diffs)

In [5]:
def Update_Code(DF, Orig_Code, Year_Diff_A, Year, Count):
    DF_Storm = DF[DF["Orig Code"] == Orig_Code].reset_index()
    Orig_Year = DF_Storm["Orig Time(Z)"][0].year
# Ignore Orig Years of 2071-2072 Due to Data Corrupsion
    if Orig_Year < 2071 or Orig_Year >= 2073:
        if Orig_Year < 2071:
            Time_Year = Orig_Year + Year_Diff_A
        elif Orig_Year >= 2073:
            Time_Year = Orig_Year + Year_Diff_A - 2
        if Time_Year == Year:
            Count += 1
        else:
            Year += 1
            Count = 1
        if Count < 10:
            Code = "TC"+str(Year)+"0"+str(Count)
        else:
            Code = "TC"+str(Year)+str(Count)
        return (Code, Year, Count)

In [6]:
def Combine_Lists(DF, Orig_List, New_List, Year_Diff, New_Code, New_Time, ABC, Orig_Combine, New_Combine, Y):
    for i in range(len(DF)):
        for l in range(len(Orig_List)):
# Update Storm Code
            if DF["Orig Code"][i] == Orig_List[l]:
                New_Code.append(New_List[l])
# Update Year
                Orig_Time = DF["Orig Time(Z)"][i]
                New_Time.append(Update_Year(Orig_Time, Year_Diff))
    for m in range(len(Orig_List)):
        ABC.append(Y)
        Orig_Combine.append(Orig_List[m])
        New_Combine.append(New_List[m])
    return (New_Code, New_Time, ABC, Orig_Combine, New_Combine)

In [7]:
def Update_Year(Orig_Time, Year_Diff):
    if Orig_Time.year < 2071:
        Year_Update = Orig_Time.year + Year_Diff
    elif Orig_Time.year >= 2073:
        Year_Update = Orig_Time.year + Year_Diff - 2
# Febuary 29 Problems
    if Orig_Time.month == 2 and Orig_Time.day == 29:
        New_Time = Orig_Time.replace(year=Year_Update, day=28)
    else:
        New_Time = Orig_Time.replace(year=Year_Update)
    return (New_Time)

In [8]:
# Open Name List File
def Open_Name_List(File):
    Name_File = pandas.read_csv(File)
    Headings = list(Name_File.columns)
    Name_List = []
    for i in range(len(Headings)):
        for j in range(len(Name_File)):
            Name_List.append(Name_File[Headings[i]][j])
    return (Name_List)

In [9]:
# Assign Names to Each Cyclone Code
def Assign_Name(Name_List, Codes, DF):
    Storm_Names = []
    for i in range(len(Codes)):
        Name = Name_List[int(i%len(Name_List))]
        Storm_Names.append(Name)
    Codes["Name"] = Storm_Names
    DF_Names = []
    for k in range(len(DF)):
        Code = DF["Code"][k]
        Name = list(Codes[Codes["New Code"] == Code]["Name"])[0]
        DF_Names.append(Name)
    DF.insert(1, "Name", DF_Names)
    return (Codes, DF)

In [10]:
Name_List = Open_Name_List('Storm_Name_List.csv')

In [11]:
Control_A_DF_Orig, Control_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.002_avg')
Control_B_DF_Orig, Control_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.003_avg')
Control_C_DF_Orig, Control_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.002_avg')

In [12]:
Control_DF_Combined, Control_Codes, Control_Year_Diffs = \
Combine_DF(Control_A_DF_Orig, Control_B_DF_Orig, Control_C_DF_Orig, \
Control_A_Storm_List_Orig, Control_B_Storm_List_Orig, Control_C_Storm_List_Orig, "Control")
Control_Codes_Names, Control_DF_Names = Assign_Name(Name_List, Control_Codes, Control_DF_Combined)

In [13]:
RCP45_A_DF_Orig, RCP45_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45_avg')
RCP45_B_DF_Orig, RCP45_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45.002_avg')
RCP45_C_DF_Orig, RCP45_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp45.003_avg')

In [14]:
RCP45_DF_Combined, RCP45_Codes, RCP45_Year_Diffs = \
Combine_DF(RCP45_A_DF_Orig, RCP45_B_DF_Orig, RCP45_C_DF_Orig, \
RCP45_A_Storm_List_Orig, RCP45_B_Storm_List_Orig, RCP45_C_Storm_List_Orig, "RCP45")
RCP45_Codes_Names, RCP45_DF_Names = Assign_Name(Name_List, RCP45_Codes, RCP45_DF_Combined)

In [15]:
RCP85_A_DF_Orig, RCP85_A_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85_avg')
RCP85_B_DF_Orig, RCP85_B_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85.002_avg')
RCP85_C_DF_Orig, RCP85_C_Storm_List_Orig = Create_DF(Diri + 'traj_et_dtime900.rcp85.003_avg')

In [16]:
RCP85_DF_Combined, RCP85_Codes, RCP85_Year_Diffs = \
Combine_DF(RCP85_A_DF_Orig, RCP85_B_DF_Orig, RCP85_C_DF_Orig, \
RCP85_A_Storm_List_Orig, RCP85_B_Storm_List_Orig, RCP85_C_Storm_List_Orig, "RCP85")
RCP85_Codes_Names, RCP85_DF_Names = Assign_Name(Name_List, RCP85_Codes, RCP85_DF_Combined)

In [17]:
#DF_Storm = RCP85_DF_Names[RCP85_DF_Names["Code"] == "TC213713"].reset_index()
#DF_Filter_Process(DF_Storm)

In [18]:
# Create DF With Extratropical Transition Information
def Create_ET_DF(DF, Codes):
    ET_Storms = []
    ET_Storm_Names = []
    for i in range(len(Codes)):
        DF_Storm_Orig = DF[DF["Code"] == Codes["New Code"][i]].reset_index()
        DF_Storm, ET_Complete = DF_Filter_Process(DF_Storm_Orig)
        if ET_Complete == True:
            ET_Storms.append(Codes["New Code"][i])
            ET_Storm_Names.append(Codes["Name"][i])
        if i == 0:
            DF_Fixed = DF_Storm.copy()
        else:
            DF_Fixed = pandas.concat([DF_Fixed, DF_Storm])
    ET_DF = pandas.DataFrame({"Code": ET_Storms, "Name": ET_Storm_Names})
    return (ET_DF, DF_Fixed)

In [19]:
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [20]:
# Check For Time Discontinuity
def Time_Discontinuity(DF_Storm):
    Time_Continuous = 0
    for k in range(len(DF_Storm)):
        if k > 0 and Time_Continuous == 0:
# If Over 12 Hours of Missing Data
            if DF_Storm["Time(Z)"][k] - DF_Storm["Time(Z)"][k-1] > datetime.timedelta(hours=12):
#                print (DF_Storm["Code"][0], k, DF_Storm["Time(Z)"][k], DF_Storm["Time(Z)"][k-1])
                Before = DF_Storm["Time(Z)"][k-1]
                After = DF_Storm["Time(Z)"][k]
                DF_Before = DF_Storm[DF_Storm["Time(Z)"] <= Before].reset_index()
                DF_After = DF_Storm[DF_Storm["Time(Z)"] >= After].reset_index()
# Choose the Part with the Lowest Min SLP to Keep
                if numpy.min(DF_Before["SLP(hPa)"]) <= numpy.min(DF_After["SLP(hPa)"]):
                    DF_Storm = DF_Before
                    Time_Continuous = 1
                else:
                    DF_Storm = DF_After
                    Time_Continuous = 1
                DF_Storm = DF_Storm.drop("level_0", axis=1)
    return (DF_Storm)

In [21]:
# Check For Track Discontinuity
def Track_Discontinuity(DF_Storm):
    Track_Continuous = 0
    for k in range(len(DF_Storm)):
        if k > 1 and Track_Continuous == 0:
# Calculate Distance Between Each 6 Hour Interval
            Distance_1 = Find_Distance(DF_Storm["Lat"][k], DF_Storm["Lat"][k-1], \
            DF_Storm["Lon"][k], DF_Storm["Lon"][k-1])
            Distance_2 = Find_Distance(DF_Storm["Lat"][k-1], DF_Storm["Lat"][k-2], \
            DF_Storm["Lon"][k-1], DF_Storm["Lon"][k-2])
# If Distance >= 500km and Distance Over 5 Times Further Than Previous Time Interval
            if Distance_1 >= 500 and Distance_1 >= Distance_2 * 5:
#                print (DF_Storm["Code"][0], k, Distance_1, Distance_2)
                Before = DF_Storm["Time(Z)"][k-1]
                After = DF_Storm["Time(Z)"][k]
                DF_Before = DF_Storm[DF_Storm["Time(Z)"] <= Before].reset_index()
                DF_After = DF_Storm[DF_Storm["Time(Z)"] >= After].reset_index()
# Choose the Part with the Lowest Min SLP to Keep
                if numpy.min(DF_Before["SLP(hPa)"]) <= numpy.min(DF_After["SLP(hPa)"]):
                    DF_Storm = DF_Before
                    Track_Continuous = 1
                else:
                    DF_Storm = DF_After
                    Track_Continuous = 1
                DF_Storm = DF_Storm.drop("level_0", axis=1)
    return (DF_Storm)

In [22]:
# Filter to Find Storm That Have Completed ET Transition
def DF_Filter_Process(DF_Storm_Orig):
    Bottom_Right = False
    Top_Right = False
    Bottom_Left = False
    Top_Left = False
    Bottom_Right_k = []
    Top_Right_k = []
    Bottom_Left_k = []
    ET_Complete = False
#
# Check For Time Discontinuity
#    print (DF_Storm_Orig["Code"][0])
#    print (len(DF_Storm_Orig))
    DF_Storm = Time_Discontinuity(DF_Storm_Orig)
#    print (len(DF_Storm))
# Repeat to Check Again
    DF_Storm = Time_Discontinuity(DF_Storm)
#    print (len(DF_Storm))
    DF_Storm = Time_Discontinuity(DF_Storm)
#    print (len(DF_Storm))
#    print (DF_Storm[20:28])
#
# Check If Track Continous
    DF_Storm = Track_Discontinuity(DF_Storm)
#    print (len(DF_Storm))
# Repeat to Check Again
    DF_Storm = Track_Discontinuity(DF_Storm)
#    print (len(DF_Storm))
    DF_Storm = Track_Discontinuity(DF_Storm)
#    print (len(DF_Storm))
#
# Check If Storm Has Completed ET Transition
    for k in range(len(DF_Storm)):
# Storm Must First Be Tropical
        if Bottom_Right == False and DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
            Bottom_Right = True
            Bottom_Right_k.append(k)
        if Bottom_Right == True and Top_Left == False:
# Storm Must Be Tropical For At Least 48 Hours
            if len(Bottom_Right_k) <= 8:
                if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
                    Bottom_Right_k.append(k)
            else:
# If Storm Ever Enters Top Left Quadrant = Completed ET
                if DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] < 0:
                    Top_Left = True
                    ET_Complete = True
# If Storm Enters Top Right Quadrant
                if DF_Storm["B"][k] > 15 and DF_Storm["VLT"][k] >= 0:
                    Top_Right = True
                    Top_Right_k.append(k)
# If Storm Enters Bottom Left Quadrant Within 24 Hours After Leaving Top Right Quadrant = Completed ET
                    if Bottom_Left == True:
                        if k-4 <= Bottom_Left_k[len(Bottom_Left_k)-1]:
                            ET_Complete = True
# As Above, But Vice Versa
                if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] < 0:
                    Bottom_Left = True
                    Bottom_Left_k.append(k)
                    if Top_Right == True:
                        if k-4 <= Top_Right_k[len(Top_Right_k)-1]:
                            ET_Complete = True
#            print (k, DF_Storm["B"][k], DF_Storm["VLT"][k], ET_Complete)
    return (DF_Storm, ET_Complete)

In [23]:
Control_ET_Orig, Control_DF_Fixed = Create_ET_DF(Control_DF_Names, Control_Codes_Names)

In [24]:
RCP45_ET_Orig, RCP45_DF_Fixed = Create_ET_DF(RCP45_DF_Names, RCP45_Codes_Names)

In [25]:
RCP85_ET_Orig, RCP85_DF_Fixed = Create_ET_DF(RCP85_DF_Names, RCP85_Codes_Names)

In [26]:
def DF_Filtered(Main_DF, ET_DF, Codes_DF):
    Code_List = ET_DF["Code"]
    Filtered_DF = Main_DF[Main_DF["Code"].isin(Code_List)].reset_index()
    Codes_Final = Codes_DF[Codes_DF["New Code"].isin(Code_List)].reset_index()
    Filtered_DF = Filtered_DF.drop("index", axis=1)
    Codes_Final = Codes_Final.drop("index", axis=1)
    return (Filtered_DF, Codes_Final)

In [27]:
Control_DF_Filtered, Control_Codes_Final = DF_Filtered(Control_DF_Fixed, Control_ET_Orig, Control_Codes_Names)
RCP45_DF_Filtered, RCP45_Codes_Final = DF_Filtered(RCP45_DF_Fixed, RCP45_ET_Orig, RCP45_Codes_Names)
RCP85_DF_Filtered, RCP85_Codes_Final = DF_Filtered(RCP85_DF_Fixed, RCP85_ET_Orig, RCP85_Codes_Names)

In [28]:
def ET_DF_Info(Main_DF, ET_DF):
    Start_Times = []
    End_Times = []
    Path_Types = []
    Main_DF_Storm_Type = []
    for i in range(len(ET_DF)):
        DF_Storm = Main_DF[Main_DF["Code"] == ET_DF["Code"][i]].reset_index()
        Start_Index, End_Index, Path_Type = Find_ET_Start_End(DF_Storm)
        Start_Time = DF_Storm["Time(Z)"][Start_Index]
        End_Time = DF_Storm["Time(Z)"][End_Index]
        Start_Times.append(Start_Time)
        End_Times.append(End_Time)
        Path_Types.append(Path_Type)
        Storm_Type = Find_Storm_Type(DF_Storm, Start_Time, End_Time)
        for k in range(len(Storm_Type)):
            Main_DF_Storm_Type.append(Storm_Type[k])
    ET_DF["Path Type"] = Path_Types
    ET_DF["Start Time"] = Start_Times
    ET_DF["End Time"] = End_Times
    Main_DF["Storm Type"] = Main_DF_Storm_Type
    return (Main_DF, ET_DF)

In [29]:
def Find_ET_Start_End(DF_Storm):
    Bottom_Right = False
    Top_Right = False
    Top_Right_Override = False
    Bottom_Left = False
    Bottom_Left_Override = False
    ET_Complete = False
    Bottom_Right_k = []
    Top_Right_k = []
    Bottom_Left_k = []
    Start_Time_k = 0
    End_Time_k = 0
    Trans_Type = 0
    for k in range(len(DF_Storm)):
        if ET_Complete == False:
            if Bottom_Right == False:
# Storm Must First Be Tropical
                if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
                    Bottom_Right = True
                    Bottom_Right_k.append(k)
            if Bottom_Right == True:
# Storm Must Be Tropical For At Least 48 Hours
                if len(Bottom_Right_k) <= 8:
                    if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
                        Bottom_Right_k.append(k)
                else:
                    if Top_Right == False and Bottom_Left == False:
                        if DF_Storm["B"][k] <= 15 and DF_Storm["VLT"][k] >= 0:
                            if len(Top_Right_k) > 0 and k-4 > Top_Right_k[len(Top_Right_k)-1]:
                                Top_Right_k = []
                            if len(Bottom_Left_k) > 0 and k-4 > Bottom_Left_k[len(Bottom_Left_k)-1]:
                                Bottom_Left_k = []
                        elif DF_Storm["B"][k] > 15:
                            Top_Right_k.append(k)
                            if len(Top_Right_k) >= 4:
                                Top_Right = True
                                Start_Time_k = Top_Right_k[0]
                            if DF_Storm["B"][k] > 30:
                                Top_Right = True
                                Top_Right_Override = True
                                Start_Time_k = Top_Right_k[0]
                            if DF_Storm["VLT"][k] < 0:
                                ET_Complete = True
                                End_Time_k = k
                                if len(Top_Right_k) > 0:
                                    Start_Time_k = Top_Right_k[0]
                                    Trans_Type = 1
                                elif len(Bottom_Left_k) > 0:
                                    Start_Time_k = Bottom_Left_k[0]
                                    Trans_Type = 2
                                elif len(Top_Right_k) == 0 and len(Bottom_Left_k) == 0:
                                        Start_Time_k = k
                                        Trans_Type = 3
                            if len(Bottom_Left_k) > 0 and k-4 <= Bottom_Left_k[len(Bottom_Left_k)-1]:
                                ET_Complete = True
                                Start_Time_k = Bottom_Left_k[0]
                                End_Time_k = k
                                Trans_Type = 2
                        elif DF_Storm["VLT"][k] < 0:
                            Bottom_Left_k.append(k)
                            if len(Bottom_Left_k) >= 4:
                                Bottom_Left = True
                                Start_Time_k = Bottom_Left_k[0]
                            if DF_Storm["VLT"][k] < -100:
                                Bottom_Left = True
                                Bottom_Left_Override = True
                                Start_Time_k = Bottom_Left_k[0]
                            if len(Top_Right_k) > 0 and k-4 <= Top_Right_k[len(Top_Right_k)-1]:
                                ET_Complete = True
                                Start_Time_k = Top_Right_k[0]
                                End_Time_k = k
                                Trans_Type = 1
                    elif Top_Right == True:
                        if DF_Storm["B"][k] > 15:
                            if DF_Storm["B"][k] > 30:
                                Top_Right_Override = True
                            if DF_Storm["VLT"][k] >= 0:
                                Top_Right_k.append(k)
                            elif DF_Storm["VLT"][k] < 0:
                                ET_Complete = True
                                End_Time_k = k
                                Trans_Type = 1
                        elif DF_Storm["B"][k] <= 15:
                            if DF_Storm["VLT"][k] >= 0:
                                if k-4 > Top_Right_k[len(Top_Right_k)-1]:
                                    if Top_Right_Override == False:
                                        Top_Right = False
                                        Top_Right_k = []
                                        Start_Time_k = 0
                            elif DF_Storm["VLT"][k] < 0:
                                if k-4 <= Top_Right_k[len(Top_Right_k)-1] or Top_Right_Override == True:
                                    ET_Complete = True
                                    Start_Time_k = Top_Right_k[0]
                                    End_Time_k = k
                                    Trans_Type = 1
                    elif Bottom_Left == True:
                        if DF_Storm["VLT"][k] < 0:
                            if DF_Storm["VLT"][k] < -100:
                                Bottom_Left_Override = True
                            if DF_Storm["B"][k] <= 15:
                                Bottom_Left_k.append(k)
                            elif DF_Storm["B"][k] > 15:
                                ET_Complete = True
                                End_Time_k = k
                                Trans_Type = 2
                        elif DF_Storm["VLT"][k] >= 0:
                            if DF_Storm["B"][k] <= 15:
                                if k-4 > Bottom_Left_k[len(Bottom_Left_k)-1]:
                                    if Bottom_Left_Override == False:
                                        Bottom_Left = False
                                        Bottom_Left_k = []
                                        Start_Time_k = 0
                            elif DF_Storm["B"][k] > 15:
                                if k-4 <= Bottom_Left_k[len(Bottom_Left_k)-1] or Bottom_Left_Override == True:
                                    ET_Complete = True
                                    Start_Time_k = Bottom_Left_k[0]
                                    End_Time_k = k
                                    Trans_Type = 2
#        print (k, DF_Storm["B"][k], DF_Storm["VLT"][k], Start_Time_k, End_Time_k, ET_Complete, \
#        Top_Right_k, Bottom_Left_k)
    if ET_Complete == False:
        print (DF_Storm["Code"][0])
    return (Start_Time_k, End_Time_k, Trans_Type)

In [30]:
def Find_Storm_Type(DF_Storm, Start_Time, End_Time):
    Time = list(DF_Storm["Time(Z)"])
    B = list(DF_Storm["B"])
    VLT = list(DF_Storm["VLT"])
    ET_Type = []
    for k in range(len(DF_Storm)):
        if Time[k] < Start_Time:
            if B[k] < 30 and VLT[k] > -100:
                    ET_Type.append("Tropical")
            else:
                ET_Type.append("Transition")
        elif Time[k] < End_Time:
            if B[k] > 0 or VLT[k] < 100:
                ET_Type.append("Transition")
            else:
                ET_Type.append("Tropical")
        else:
            if B[k] > 0 and VLT[k] < 100:
                ET_Type.append("Extratropical")
            else:
                ET_Type.append("Transition")
    return (ET_Type)

In [31]:
Control_DF_Final, Control_ET_Timings = ET_DF_Info(Control_DF_Filtered, Control_ET_Orig)
RCP45_DF_Final, RCP45_ET_Timings = ET_DF_Info(RCP45_DF_Filtered, RCP45_ET_Orig)
RCP85_DF_Final, RCP85_ET_Timings = ET_DF_Info(RCP85_DF_Filtered, RCP85_ET_Orig)

In [32]:
#DF_Storm = Control_DF_Final[Control_DF_Final["Code"] == "TC191608"].reset_index()
#DF_Storm = RCP45_DF_Final[RCP45_DF_Final["Code"] == "TC200409"].reset_index()
#DF_Storm = RCP85_DF_Final[RCP85_DF_Final["Code"] == "TC216408"].reset_index()
#Find_ET_Start_End(DF_Storm)

In [33]:
Control_DF_Final[265:277]

,level_0,Code,Name,Lon,Lat,SLP(hPa),Winds(m/s),B,VLT,VUT,Time(Z),Storm Type
265,62,TC190105,Lydia,-56.33,37.40,948.48,41.5,9.05,236.10,289.92,1901-09-27 00:00:00,Tropical
266,63,TC190105,Lydia,-54.83,38.25,950.98,39.9,11.90,234.39,267.61,1901-09-27 06:00:00,Tropical
267,64,TC190105,Lydia,-53.37,39.19,953.76,43.1,18.13,224.89,230.05,1901-09-27 12:00:00,Transition
268,65,TC190105,Lydia,-51.56,40.08,952.91,42.0,31.98,231.25,195.68,1901-09-27 18:00:00,Transition
269,66,TC190105,Lydia,-49.70,42.10,951.29,38.4,43.81,253.98,172.97,1901-09-28 00:00:00,Transition
270,67,TC190105,Lydia,-49.51,44.55,948.89,44.0,41.29,241.12,161.34,1901-09-28 06:00:00,Transition
271,68,TC190105,Lydia,-49.52,44.98,949.44,43.2,38.45,220.08,156.22,1901-09-28 12:00:00,Transition
272,69,TC190105,Lydia,-48.37,45.66,952.82,37.7,28.80,201.31,169.85,1901-09-28 18:00:00,Transition
273,70,TC190105,Lydia,-47.67,46.28,959.65,33.9,14.50,153.80,168.18,1901-09-29 00:00:00,Transition
274,71,TC190105,Lydia,-46.92,46.83,963.87,31.0,4.41,76.01,151.33,1901-09-29 06:00:00,Transition


In [34]:
# Add ET Transition Lat Lon Into ET DataFrame
def ET_Lat_Lon(Main_DF, ET_DF):
    Array = numpy.zeros((7, len(ET_DF)))
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Start_Time = ET_DF["Start Time"][i]
        End_Time = ET_DF["End Time"][i]
        Storm = Main_DF[Main_DF["Code"] == Code]
        Array[0][i] = numpy.min(Storm["SLP(hPa)"])
        Array[1][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["SLP(hPa)"].iloc[0])
        Array[2][i] = float(Storm[Storm["Time(Z)"] == End_Time]["SLP(hPa)"].iloc[0])
        Array[3][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["Lon"].iloc[0])
        Array[4][i] = float(Storm[Storm["Time(Z)"] == Start_Time]["Lat"].iloc[0])
        Array[5][i] = float(Storm[Storm["Time(Z)"] == End_Time]["Lon"].iloc[0])
        Array[6][i] = float(Storm[Storm["Time(Z)"] == End_Time]["Lat"].iloc[0])
    ET_DF["Min SLP"] = Array[0]
    ET_DF["Start SLP"] = Array[1]
    ET_DF["End SLP"] = Array[2]
    ET_DF["Start Lon"] = Array[3]
    ET_DF["Start Lat"] = Array[4]
    ET_DF["End Lon"] = Array[5]
    ET_DF["End Lat"] = Array[6]
    return (ET_DF)

In [35]:
# Calculate Distance Between ET Start and End Points
def ET_Distance(ET_DF):
    Array = numpy.zeros(len(ET_DF))
    for i in range(len(ET_DF)):
        Array[i] = Find_Distance(ET_DF["Start Lat"][i], ET_DF["End Lat"][i], \
        ET_DF["Start Lon"][i], ET_DF["End Lon"][i])
    ET_DF["ET Dist (km)"] = Array
    return (ET_DF)

In [36]:
# Calculate Duration Between Storm Formation and Start of ET Transition
def ET_Durations(Main_DF, ET_DF):
    Array = numpy.zeros((2,len(ET_DF)))
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Storm = Main_DF[Main_DF["Code"] == Code]
        Birth_Time = list(Storm["Time(Z)"])[0]
        Start_Time = ET_DF["Start Time"][i]
        End_Time = ET_DF["End Time"][i]
        Trop_Time_Diff = Start_Time - Birth_Time
        Trop_Hours = int(Trop_Time_Diff.total_seconds()) / 3600
        Array[0][i] = Trop_Hours
        ET_Time_Diff = End_Time - Start_Time
        ET_Hours = int(ET_Time_Diff.total_seconds()) / 3600
        Array[1][i] = ET_Hours
    ET_DF["ET Duration (hr)"] = Array[1]
    ET_DF["Trop Duration (hr)"] = Array[0]
    return (ET_DF)

In [37]:
def ET_Variables(Final_DF, ET_DF):
    ET_DF = ET_Lat_Lon(Final_DF, ET_DF)
    ET_DF = ET_Distance(ET_DF)
    ET_DF = ET_Durations(Final_DF, ET_DF)
    return (ET_DF)

In [38]:
Control_ET_Final = ET_Variables(Control_DF_Final, Control_ET_Timings)
RCP45_ET_Final = ET_Variables(RCP45_DF_Final, RCP45_ET_Timings)
RCP85_ET_Final = ET_Variables(RCP85_DF_Final, RCP85_ET_Timings)

In [39]:
Control_ET_Final[5:10]

,Code,Name,Path Type,Start Time,End Time,Min SLP,Start SLP,End SLP,Start Lon,Start Lat,End Lon,End Lat,ET Dist (km),ET Duration (hr),Trop Duration (hr)
5,TC190105,Lydia,1,1901-09-27 12:00:00,1901-09-29 18:00:00,913.33,953.76,970.00,-53.37,39.19,-42.82,48.53,1335.846025,54.0,384.0
6,TC190106,Mario,1,1901-09-22 00:00:00,1901-09-22 18:00:00,988.80,993.69,991.98,-78.47,35.78,-71.20,41.68,909.539822,18.0,72.0
7,TC190107,Nicole,2,1901-10-14 12:00:00,1901-10-15 00:00:00,974.56,1015.94,1019.52,-22.00,42.00,-26.00,41.25,343.074813,12.0,246.0
8,TC190109,Shannon,1,1901-12-14 00:00:00,1901-12-14 06:00:00,942.03,975.20,975.97,-32.75,47.00,-30.25,47.75,206.101857,6.0,90.0
9,TC190201,Tony,2,1902-01-27 18:00:00,1902-01-28 00:00:00,1002.71,1012.65,1014.76,-43.50,29.50,-44.50,28.00,193.395015,6.0,60.0


In [40]:
# Output File
def Output_File(DF, Model, Type):
    File_Name = str(Model+'_'+Type+'_Output_V3.csv')
    Output_Diri = '/glade/u/home/whimkao//ExtraTrack/Output_Files/'
    DF.to_csv(Output_Diri+File_Name)

In [41]:
Output_File(Control_DF_Final, "Control", "Data")
Output_File(Control_ET_Final, "Control", "ET")
Output_File(Control_Codes_Final, "Control", "Codes")

In [42]:
Output_File(RCP45_DF_Final, "RCP45", "Data")
Output_File(RCP45_ET_Final, "RCP45", "ET")
Output_File(RCP45_Codes_Final, "RCP45", "Codes")

In [43]:
Output_File(RCP85_DF_Final, "RCP85", "Data")
Output_File(RCP85_ET_Final, "RCP85", "ET")
Output_File(RCP85_Codes_Final, "RCP85", "Codes")